In [1]:
import pymysql

## 테이블 생성

In [3]:
queries = [
    '''
    CREATE TABLE BUILDING_KILL (
        match_id BIGINT NOT NULL,
        assist_id INT NULL,
        bounty INT NULL,
        building_type VARCHAR(20) NULL,
        participant_id INT NULL,
        line_type VARCHAR(20) NULL,
        team_id INT NULL,
        timestamp INT NULL,
        tower_type VARCHAR(20) NULL
    );''',
    '''
    CREATE TABLE WARD_PLACED (
        match_id BIGINT NOT NULL,
        participant_id INT NULL,
        timestamp INT NULL,
        ward_type VARCHAR(20) NULL
    );''',
    '''
    CREATE TABLE ELITE_MONSTER_KILL (
        match_id BIGINT NOT NULL,
        assist_id INT NULL,
        bounty INT NULL,
        participant_id INT NULL,
        monster_subtype VARCHAR(20) NULL,
        monster_type VARCHAR(20) NULL,
        timestamp INT NULL
    );''',
    '''
    CREATE TABLE TURRET_PLATE_DESTROYED (
        match_id BIGINT NOT NULL,
        participant_id INT NULL,
        line_type VARCHAR(20) NULL,
        team_id INT NULL,
        timestamp INT NULL
    );''',
    '''
    CREATE TABLE OBJECTIVE_BOUNTY_PRESTART (
        match_id BIGINT NOT NULL,
        actual_starttime INT NULL,
        team_id INT NULL,
        timestamp INT NULL
    );''',
    '''
    CREATE TABLE CHAMPION_KILL (
        match_id BIGINT NOT NULL,
        assist_id INT NULL,
        bounty INT NULL,
        killstreak_length INT NULL,
        participant_id INT NULL,
        shutdown_bounty INT NULL,
        timestamp INT NULL,
        victim_id INT NULL
    );''',
    '''
    CREATE TABLE CHAMPION_SPECIAL_KILL (
        match_id BIGINT NOT NULL,
        kill_type VARCHAR(20) NULL,
        participant_id INT NULL,
        multikil_length INT NULL,
        timestamp INT NULL
    );''',
    '''
    CREATE TABLE ITEM_DESTROYED (
        match_id BIGINT NOT NULL,
        item_id INT NULL,
        participant_id INT NULL,
        timestamp INT NULL
    );''',
    '''
    CREATE TABLE ITEM_PURCHASED (
        match_id BIGINT NOT NULL,
        item_id INT NULL,
        participant_id INT NULL,
        timestamp INT NULL
    );'''
]

In [5]:
conn = pymysql.connect(
    host="localhost",
    user='root',
    password='1221'
)

In [7]:
cur = conn.cursor()

In [9]:
cur.execute('use LOL_data;')

0

In [11]:
for q in queries:
    cur.execute(q)

In [13]:
cur.close()
conn.close()

## 데이터 삽입

In [15]:
import json

path = 'KR_7342260205.json'

with open(path, 'r', encoding='utf-8') as file:
    match_tl = json.load(file)

### DB 연결

In [17]:
conn = pymysql.connect(
    host="localhost",
    user='root',
    password='1221'
)

conn.autocommit(False)
cur = conn.cursor()
cur.execute('use LOL_data;')

0

### 데이터 삽입 쿼리 함수 정의

In [19]:
building_kill = 'BUILDING_KILL'
ward_placed = 'WARD_PLACED'
elite_monster_kill = 'ELITE_MONSTER_KILL'
turret_plate_destroyed = 'TURRET_PLATE_DESTROYED'
objective_bounty_prestart = 'OBJECTIVE_BOUNTY_PRESTART'
champion_kill = 'CHAMPION_KILL'
champion_special_kill = 'CHAMPION_SPECIAL_KILL'
item_destroyed = 'ITEM_DESTROYED'
item_purchased = 'ITEM_PURCHASED'

In [21]:
def assist_id_list2int(data): # assist ID의 리스트를 integer로 변환
    data = [1 if x in data else 0 for x in range(11)]
    return int(''.join(map(str, data)), 2)

def get_assist_id_list(data): # integer를 assist ID의 리스트로 변환
    data = ''.join(['0' for x in range(11 - len(bin(data)[2:]))]) + bin(data)[2:]
    return [i for i in range(len(data)) if data[i] == '1']

In [107]:
def query_insert_buildingKill(event, matchID):
    if 'assistingParticipantIds' in event.keys():
        event['assist_id'] = assist_id_list2int(event['assistingParticipantIds'])
    else: event['assist_id'] = 'null'

    if 'towerType' not in event.keys(): event['towerType'] = 'null'
        
    return f'''insert into {building_kill} (match_id, assist_id, bounty, building_type, participant_id, line_type, team_id, timestamp, tower_type)
            values ({matchID}, {event['assist_id']}, {event['bounty']}, '{event['buildingType']}', {event['killerId']}, '{event['laneType']}',
            {event['teamId']}, {event['timestamp']}, '{event['towerType']}');'''

In [53]:
def query_insert_wardPlaced(event, matchID):
    return f'''insert into {ward_placed} (match_id, participant_id, timestamp, ward_type)
            values ({matchID}, {event['creatorId']}, {event['timestamp']}, '{event['wardType']}');'''

In [83]:
def query_insert_eliteMonsterKill(event, matchID):
    if 'assistingParticipantIds' in event.keys():
        event['assist_id'] = assist_id_list2int(event['assistingParticipantIds'])
    else: event['assist_id'] = 'null'

    if 'monsterSubType' not in event.keys():
        event['monsterSubType'] = 'null'

    return f'''insert into {elite_monster_kill} (match_id, assist_id, bounty, participant_id, monster_subtype, monster_type, timestamp)
            values ({matchID}, {event['assist_id']}, {event['bounty']}, {event['killerId']}, '{event['monsterSubType']}', '{event['monsterType']}',
            {event['timestamp']});'''

In [57]:
def query_insert_turretPlateDestroyed(event, matchID):
    return f'''insert into {turret_plate_destroyed} (match_id, participant_id, line_type, team_id, timestamp)
            values ({matchID}, {event['killerId']}, '{event['laneType']}', {event['teamId']}, {event['timestamp']});'''

In [31]:
def query_insert_objectiveBountyPrestart(event, matchID):
    return f'''insert into {objective_bounty_prestart} (match_id, actual_starttime, team_id, timestamp)
            values ({matchID}, {event['actualStartTime']}, {event['teamId']}, {event['timestamp']});'''

In [85]:
def query_insert_championKill(event, matchID):
    if 'assistingParticipantIds' in event.keys():
        event['assist_id'] = assist_id_list2int(event['assistingParticipantIds'])
    else: event['assist_id'] = 'null'

    return f'''insert into {champion_kill} (match_id, assist_id, bounty, killstreak_length, participant_id, shutdown_bounty, timestamp, victim_id)
            values ({matchID}, {event['assist_id']}, {event['bounty']}, {event['killStreakLength']}, {event['killerId']}, {event['shutdownBounty']},
            {event['timestamp']}, {event['victimId']});'''

In [99]:
def query_insert_championSpecialKill(event, matchID):
    if 'multiKillLength' not in event.keys() or event['multiKillLength'] == None:
        event['multiKillLength'] = 'null'
        
    return f'''insert into {champion_special_kill} (match_id, kill_type, participant_id, multikil_length, timestamp)
            values ({matchID}, '{event['killType']}', {event['killerId']}, {event['multiKillLength']}, {event['timestamp']});'''

In [37]:
def query_insert_itemDestroyed(event, matchID):
    return f'''insert into {item_destroyed} (match_id, item_id, participant_id, timestamp)
            values ({matchID}, {event['itemId']}, {event['participantId']}, {event['timestamp']});'''

In [39]:
def query_insert_itemPurchased(event, matchID):
    return f'''insert into {item_purchased} (match_id, item_id, participant_id, timestamp)
            values ({matchID}, {event['itemId']}, {event['participantId']}, {event['timestamp']});'''

### 데이터 삽입 함수 정의

In [71]:
def insert_events(match_tl, cur):
    matchID = int(match_tl['metadata']['matchId'][3:])
    for frame in match_tl['info']['frames']:
        for event in frame['events']:
            print(event['type'])
            query = None
            if event['type'] == building_kill:
                query = query_insert_buildingKill(event, matchID)
            if event['type'] == ward_placed:
                query = query_insert_wardPlaced(event, matchID)
            if event['type'] == elite_monster_kill:
                query = query_insert_eliteMonsterKill(event, matchID)
            if event['type'] == turret_plate_destroyed:
                query = query_insert_turretPlateDestroyed(event, matchID)
            if event['type'] == objective_bounty_prestart:
                query = query_insert_objectiveBountyPrestart(event, matchID)
            if event['type'] == champion_kill:
                query = query_insert_championKill(event, matchID)
            if event['type'] == champion_special_kill:
                query = query_insert_championSpecialKill(event, matchID)
            if event['type'] == item_destroyed:
                query = query_insert_itemDestroyed(event, matchID)
            if event['type'] == item_purchased:
                query = query_insert_itemPurchased(event, matchID)

            if query:
                print(query)
                cur.execute(query)

In [105]:
for frame in match_tl['info']['frames']:
    for event in frame['events']:
        if event['type'] == building_kill:
            print(event)
            print('towerType' in event.keys())
    #print(match_tl['info']['frames'][int(frame)]['events'])

{'bounty': 0, 'buildingType': 'TOWER_BUILDING', 'killerId': 6, 'laneType': 'TOP_LANE', 'position': {'x': 981, 'y': 10441}, 'teamId': 100, 'timestamp': 750187, 'towerType': 'OUTER_TURRET', 'type': 'BUILDING_KILL', 'assist_id': 'null'}
True
{'bounty': 0, 'buildingType': 'TOWER_BUILDING', 'killerId': 0, 'laneType': 'BOT_LANE', 'position': {'x': 10504, 'y': 1029}, 'teamId': 100, 'timestamp': 894075, 'towerType': 'OUTER_TURRET', 'type': 'BUILDING_KILL', 'assist_id': 'null'}
True
{'assistingParticipantIds': [9], 'bounty': 0, 'buildingType': 'TOWER_BUILDING', 'killerId': 7, 'laneType': 'MID_LANE', 'position': {'x': 5846, 'y': 6396}, 'teamId': 100, 'timestamp': 927806, 'towerType': 'OUTER_TURRET', 'type': 'BUILDING_KILL', 'assist_id': 2}
True
{'bounty': 0, 'buildingType': 'TOWER_BUILDING', 'killerId': 0, 'laneType': 'BOT_LANE', 'position': {'x': 13866, 'y': 4505}, 'teamId': 200, 'timestamp': 1130820, 'towerType': 'OUTER_TURRET', 'type': 'BUILDING_KILL', 'assist_id': 'null'}
True
{'bounty': 0, 

### 삽입 해보기

In [111]:
insert_events(match_tl, cur)

PAUSE_END
ITEM_PURCHASED
insert into ITEM_PURCHASED (match_id, item_id, participant_id, timestamp)
            values (7342260205, 1055, 6, 1820);
ITEM_PURCHASED
insert into ITEM_PURCHASED (match_id, item_id, participant_id, timestamp)
            values (7342260205, 2003, 6, 1820);
ITEM_PURCHASED
insert into ITEM_PURCHASED (match_id, item_id, participant_id, timestamp)
            values (7342260205, 1056, 8, 2488);
ITEM_PURCHASED
insert into ITEM_PURCHASED (match_id, item_id, participant_id, timestamp)
            values (7342260205, 2003, 8, 3089);
ITEM_PURCHASED
insert into ITEM_PURCHASED (match_id, item_id, participant_id, timestamp)
            values (7342260205, 2003, 8, 3190);
ITEM_PURCHASED
insert into ITEM_PURCHASED (match_id, item_id, participant_id, timestamp)
            values (7342260205, 3340, 8, 3323);
ITEM_PURCHASED
insert into ITEM_PURCHASED (match_id, item_id, participant_id, timestamp)
            values (7342260205, 1101, 7, 3490);
ITEM_PURCHASED
insert into ITEM

In [113]:
conn.commit()

In [109]:
conn.rollback()